# Wissensaggregator Mittelalter und frühe Neuzeit
WIAGweb2

## Daten für Bischöfe und Domherren aus verschiedenen Quellen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben oder als CSV-Datei exportiert. Vergleiche die Daten in Bezug auf GSN, Namen, Lebensdaten

[Domherren](#Domherren)  
[Bischöfe](#Bischöfe)  
[Personendatenbank](#Personendatenbank)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [83]:
out_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data"

In [5]:
using Pkg

In [6]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [7]:
using Revise

In [8]:
using WiagDataSetup

In [9]:
using MySQL, DataFrames

In [10]:
ENV["COLUMNS"] = 120

120

In [11]:
Wds = WiagDataSetup

WiagDataSetup

In [12]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

Item Type ID

In [13]:
item_type_id = 5

5

In [25]:
data_path_dh = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-05-18"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-05-18"

In [26]:
data_path_ep = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-05-18"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-05-18"

## Domherren

In [14]:
using CSV

In [49]:
# Quelle CSV-Datei
filename = joinpath(data_path_dh, "tbl_domherren.csv")
df_dh = CSV.read(filename, DataFrame);

Leerzeichen bereinigen

In [50]:
Wds.clean_up!(df_dh)

In [51]:
first(df_dh, 5)

,ID_Domherr,Eintrag_Quelle,Seite_Quelle,Quelle,Praefix,Vorname,Familienname,Sterbedatum,Geburtsdatum
,Int64,String?,String?,Int64,String?,String?,String?,String?,String?
1,10000,missing,"167, 183",6,von,Friedrich Wilhelm,Loe,1689,missing
2,10001,missing,183,6,von,Johann Heinrich,Metzenhausen,missing,missing
3,10002,missing,183,6,von,Philipp Friedrich,Metzenhausen,missing,missing
4,10003,missing,183,6,von der,Damian Friedrich,Fels,1684,missing
5,10004,missing,"183, 189",6,von,Friedrich Ernst,Eltz-Rodendorf,missing,missing


*2022-03-10* prüfe auf unorthodoxe Einräge in `WIAG_ID_Bischof`

In [52]:
df_dh_ID_Bischof = dropmissing(df_dh, :WIAG_ID_Bischof);

In [53]:
df_dh_ID_Bischof[201:205, [:ID_Domherr, :Vorname, :Familienname, :WIAG_ID_Bischof, :Status]]

,ID_Domherr,Vorname,Familienname,WIAG_ID_Bischof,Status
,Int64,String?,String?,String,String?
1,14815,Nikolaus,Riesenburg,WIAG-Pers-EPISCGatz-03242-001,fertig
2,14816,Albrecht,Blarer,WIAG-Pers-EPISCGatz-02920-001,fertig
3,14818,Konrad,Rechberg zu Hohenrechberg,WIAG-Pers-EPISCGatz-02704-001,fertig
4,14819,Thomas,Berlower,WIAG-Pers-EPISCGatz-05308-001,fertig
5,14827,Johann Franz Anton,Khevenhueller,WIAG-Pers-EPISCGatz-10370-001,fertig


In [54]:
find_spurious(a) = !ismissing(a) && occursin("http", a)
df_dh_sp = subset(df_dh, :WIAG_ID_Bischof => ByRow(find_spurious));

In [55]:
# 2022-05-18
size(df_cn_sp)

(0, 48)

In [44]:
function get_ID_Bischof(s)
    if ismissing(s)
        return s
    end
    rgx = r"WIAG-Pers-EPISCGatz-([0-9]+)-00?"
    rgm = match(rgx, s)
    if isnothing(rgm)
        @warn "No match in ", s
        return missing
    end
    return parse(Int, rgm[1])
end


get_ID_Bischof (generic function with 1 method)

In [45]:
get_ID_Bischof("WIAG-Pers-EPISCGatz-03242-001")

3242

In [56]:
transform!(df_dh, :WIAG_ID_Bischof => ByRow(get_ID_Bischof) => :ID_Bischof);

## Bischöfe

In [14]:
using CSV

In [28]:
# Quelle CSV-Datei
filename = joinpath(data_path_ep, "tbl_Bischoefe_Gatz.csv")
df_ep = CSV.read(filename, DataFrame);

Leerzeichen bereinigen

In [29]:
Wds.clean_up!(df_ep)

In [30]:
first(df_ep, 5)

,ID_Bischof,Eintrag RI OPAC,Autoren Gatz,Seite Gatz,Praefix,Vorname,Familienname
,Int64,String,String?,String,String?,String,String?
1,21086,"um 1372, Johannes",missing,137,missing,Johannes,missing
2,21087,"um 1378, Michael, Ep. Sythiensis",missing,137,missing,Michael,missing
3,21088,"1392-1398, Fr. Dietricus, OFM, Ep. Siginensis",missing,137,missing,Dietricus,missing
4,21089,"1406-1408, Bertholdus, OP, Ep. Sewastopolensis",missing,137,missing,Bertholdus,missing
5,21090,"1411-1418, Conradus Simplicy, OFM, Ep. Siginensis",missing,137,missing,Conradus,Simplicy


In [58]:
names(df_dh)[20:end]

30-element Vector{String}:
 "GSN_ID"
 "VIAF_ID"
 "Wikidata_ID"
 "Autor_Original"
 "Familienname_Variante"
 "Vorname_Variante"
 "Kommentar_Name"
 "Kommentar_Person"
 "QRang_GND"
 "QRang_GS"
 "Qualitaet_GS"
 "Qualitaet_GND"
 "BelegdatumBeginn"
 ⋮
 "Normdaten recherchiert"
 "URL_Wikipedia"
 "Externe URL"
 "ID_Quelle"
 "ID_Domherr_Jasmin"
 "ID_Fremdschluessel_2"
 "Merged_Into"
 "Identisch_mit"
 "WIAG_ID_Bischof"
 "LOD_Academy"
 "ID_Fremdschlüssel_Num"
 "ID_Bischof"

In [70]:
columns_dh = [
    :ID_Domherr => :ID_Domherr,
    :ID_Bischof => :ID_Bischof,
    :Vorname => :Vorname_dh,
    :Familienname => :Familienname_dh,
    :Sterbedatum => :Sterbedatum_dh,
    :Geburtsdatum => :Geburtsdatum_dh,
    :Vorname_Variante => :Vorname_Variante_dh,
    :Familienname_Variante => :Familienname_Variante_dh,    
    :GSN_ID => :GSN_dh
];

In [66]:
columns_ep = [
    :ID_Bischof => :ID_Bischof,
    :Vorname => :Vorname_ep,
    :Familienname => :Familienname_ep,
    :Sterbedatum => :Sterbedatum_ep,
    :Geburtsdatum => :Geburtsdatum_ep,
    :Vorname_Varianten => :Vorname_Variante_ep,
    :Familienname_Variante => :Familienname_Variante_ep,
    :GSN_ID => :GSN_ep
];

In [67]:
select(df_ep[1:5, :], columns_ep)

,ID_Bischof,Vorname_ep,Familienname_ep,Sterbedatum_ep,Geburtsdatum_ep,Vorname_Variante_ep,Familienname_Variante_ep
,Int64,String,String?,String?,String?,String?,String?
1,21086,Johannes,missing,missing,missing,missing,missing
2,21087,Michael,missing,missing,missing,missing,missing
3,21088,Dietricus,missing,missing,missing,missing,missing
4,21089,Bertholdus,missing,missing,missing,missing,missing
5,21090,Conradus,Simplicy,missing,missing,missing,missing


In [75]:
df_dh_ep = innerjoin(select(df_dh, columns_dh), select(df_ep, columns_ep), on = :ID_Bischof, matchmissing = :notequal);

In [76]:
size(df_dh_ep)

(836, 16)

In [78]:
delta(a, b) = (!ismissing(a) && !ismissing(b) && a != b) 

delta (generic function with 1 method)

In [82]:
transform!(df_dh_ep, [:GSN_dh, :GSN_ep] => ByRow(delta) => :delta_GSN);

In [86]:
transform!(df_dh_ep, [:Vorname_dh, :Vorname_ep] => ByRow(delta) => :delta_Vorname);

In [87]:
transform!(df_dh_ep, [:Familienname_dh, :Familienname_ep] => ByRow(delta) => :delta_Familienname);

In [102]:
transform!(df_dh_ep, [:Geburtsdatum_dh, :Geburtsdatum_ep] => ByRow(delta) => :delta_Geburtsdatum);

In [103]:
transform!(df_dh_ep, [:Sterbedatum_dh, :Sterbedatum_ep] => ByRow(delta) => :delta_Sterbedatum);

In [104]:
mult_or(p, q, r, s, t) = p || q || r || s || t

mult_or (generic function with 2 methods)

In [106]:
transform!(df_dh_ep, [:delta_GSN, :delta_Vorname, :delta_Familienname, :delta_Geburtsdatum, :delta_Sterbedatum] => ByRow(mult_or) => :delta_filter);

In [107]:
df_dh_ep_delta = subset(df_dh_ep, :delta_filter);

In [108]:
size(df_dh_ep_delta)

(461, 22)

In [109]:
filename = joinpath(out_path, "delta_dh_ep.csv")
CSV.write(filename, df_dh_ep_delta, delim = ";")

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\delta_dh_ep.csv"

Sortiere die Spalten nach Inhalt statt nach Quelle

In [110]:
names(df_dh_ep_delta)

22-element Vector{String}:
 "ID_Domherr"
 "ID_Bischof"
 "Vorname_dh"
 "Familienname_dh"
 "Sterbedatum_dh"
 "Geburtsdatum_dh"
 "Vorname_Variante_dh"
 "Familienname_Variante_dh"
 "GSN_dh"
 "Vorname_ep"
 "Familienname_ep"
 "Sterbedatum_ep"
 "Geburtsdatum_ep"
 "Vorname_Variante_ep"
 "Familienname_Variante_ep"
 "GSN_ep"
 "delta_GSN"
 "delta_Vorname"
 "delta_Familienname"
 "delta_filter"
 "delta_Geburtsdatum"
 "delta_Sterbedatum"

In [118]:
columns_c = [
    :delta_filter,
    :delta_GSN,
    :delta_Vorname,
    :delta_Familienname,
    :delta_Geburtsdatum,
    :delta_Sterbedatum,
    :ID_Domherr,
    :ID_Bischof,
    :GSN_dh,
    :GSN_ep,
    :Vorname_dh,
    :Familienname_dh,
    :Vorname_ep,
    :Familienname_ep,
    :Geburtsdatum_dh,
    :Sterbedatum_dh,
    :Geburtsdatum_ep,
    :Sterbedatum_ep,
    :Vorname_Variante_dh,
    :Vorname_Variante_ep,
    :Familienname_Variante_dh,
    :Familienname_Variante_ep,
];

In [119]:
filename = joinpath(out_path, "delta_dh_ep_cs_1.csv")
CSV.write(filename, select(df_dh_ep_delta, columns_c), delim = ";")

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\delta_dh_ep_cs_1.csv"